# Practical Exam: House sales

RealAgents is a real estate company that focuses on selling houses.

RealAgents sells a variety of types of house in one metropolitan area.

Some houses sell slowly and sometimes require lowering the price in order to find a buyer.

In order to stay competitive, RealAgents would like to optimize the listing prices of the houses it is trying to sell.

They want to do this by predicting the sale price of a house given its characteristics.

If they can predict the sale price in advance, they can decrease the time to sale.


## Data

The dataset contains records of previous houses sold in the area.

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| house_id    | Nominal. </br> Unique identifier for houses. </br>Missing values not possible. |
| city        | Nominal. </br>The city in which the house is located. One of 'Silvertown', 'Riverford', 'Teasdale' and 'Poppleton'. </br>Replace missing values with "Unknown". |
| sale_price  | Discrete. </br>The sale price of the house in whole dollars. Values can be any positive number greater than or equal to zero.</br>Remove missing entries. |
| sale_date   | Discrete. </br>The date of the last sale of the house. </br>Replace missing values with 2023-01-01. |
| months_listed  | Continuous. </br>The number of months the house was listed on the market prior to its last sale, rounded to one decimal place. </br>Replace missing values with mean number of months listed, to one decimal place. |
| bedrooms    | Discrete. </br>The number of bedrooms in the house. Any positive values greater than or equal to zero. </br>Replace missing values with the mean number of bedrooms, rounded to the nearest integer. |
| house_type   | Ordinal. </br>One of "Terraced" (two shared walls), "Semi-detached" (one shared wall), or "Detached" (no shared walls). </br>Replace missing values with the most common house type. |
| area      | Continuous. </br>The area of the house in square meters, rounded to one decimal place. </br>Replace missing values with the mean, to one decimal place. |


# Task 1

The team at RealAgents knows that the city that a property is located in makes a difference to the sale price. 

Unfortuntately they believe that this isn't always recorded in the data. 

Calculate the number of missing values of the `city`. 

 - You should use the data in the file "house_sales.csv". 

 - Your output should be an object `missing_city`, that contains the number of missing values in this column. 

In [1]:
# Use this cell to write your code for Task 1
# import libraries
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Load the dataset
df = pd.read_csv("house_sales.csv")
df.head()

# Identify and replace missing values with 'Unknown'
missing_city = (df['city'] == '--').sum()
print(f"The total number of missing cities represented by '--' is {missing_city}")

The total number of missing cities represented by '--' is 73


# Task 2 

Before you fit any models, you will need to make sure the data is clean. 

The table below shows what the data should look like. 

Create a cleaned version of the dataframe. 

 - You should start with the data in the file "house_sales.csv". 

 - Your output should be a dataframe named `clean_data`. 

 - All column names and values should match the table below.


| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| house_id    | Nominal. </br> Unique identifier for houses. </br>Missing values not possible. |
| city        | Nominal. </br>The city in which the house is located. One of 'Silvertown', 'Riverford', 'Teasdale' and 'Poppleton' </br>Replace missing values with "Unknown". |
| sale_price  | Discrete. </br>The sale price of the house in whole dollars. Values can be any positive number greater than or equal to zero.</br>Remove missing entries. |
| sale_date   | Discrete. </br>The date of the last sale of the house. </br>Replace missing values with 2023-01-01. |
| months_listed  | Continuous. </br>The number of months the house was listed on the market prior to its last sale, rounded to one decimal place. </br>Replace missing values with mean number of months listed, to one decimal place. |
| bedrooms    | Discrete. </br>The number of bedrooms in the house. Any positive values greater than or equal to zero. </br>Replace missing values with the mean number of bedrooms, rounded to the nearest integer. |
| house_type   | Ordinal. </br>One of "Terraced", "Semi-detached", or "Detached". </br>Replace missing values with the most common house type. |
| area      | Continuous. </br>The area of the house in square meters, rounded to one decimal place. </br>Replace missing values with the mean, to one decimal place. |

In [2]:
# Use this cell to write your code for Task 2

# Replacing the missing values '--' in the city with 'Unknown'
df['city'] = df['city'].replace('--', 'Unknown')

# Dropping missing values in sale_price
df = df.dropna(subset=['sale_price'])

# Replacing missing values in sale_date with 2023-01-01.
df['sale_date'] = pd.to_datetime(df['sale_date'].fillna('2023-01-01'))

# Replacing missing values with the average number of months listed, to one decimal place 
mean_months = round(df['months_listed'].mean(), 1)
df['months_listed'] = df['months_listed'].fillna(mean_months) 

# Replacing missing values with the mean number of bedrooms, rounded to the nearest integer
mean_bedrooms = round(df['bedrooms'].mean())
df['bedrooms'] = df['bedrooms'].fillna(mean_bedrooms)

# Standardizing the values and then replacing missing values with the most common house type
df['house_type'] = df['house_type'].replace({
    'Det.': 'Detached',
    'Semi': 'Semi-detached',
    'Terr.': 'Terraced'
})
frequent_type = df['house_type'].mode()[0]
df['house_type'] = df['house_type'].fillna(frequent_type)

# Stripping off the "sq.m", then converting to float data type and finally Replacing missing values with the mean, to one decimal place
df['area'] = df['area'].astype(str).str.replace(' sq.m.', '', regex=False).astype(float)

mean_area = round(df['area'].mean(), 1)
df['area'] = df['area'].fillna(mean_area)

# Assigning the final dataframe a name 'clean_data'
clean_data = df
clean_data.head()

,house_id,city,sale_price,sale_date,months_listed,bedrooms,house_type,area
0,1217792,Silvertown,55943,2021-09-12,5.4,2,Semi-detached,107.8
1,1900913,Silvertown,384677,2021-01-17,6.3,5,Detached,498.8
2,1174927,Riverford,281707,2021-11-10,6.9,6,Detached,542.5
3,1773666,Silvertown,373251,2020-04-13,6.1,6,Detached,528.4
4,1258487,Silvertown,328885,2020-09-24,8.7,5,Detached,477.1


# Task 3 

The team at RealAgents have told you that they have always believed that the number of bedrooms is the biggest driver of house price. 

Producing a table showing the difference in the average sale price by number of bedrooms along with the variance to investigate this question for the team.

 - You should start with the data in the file 'house_sales.csv'.

 - Your output should be a data frame named `price_by_rooms`. 

 - It should include the three columns `bedrooms`, `avg_price`, `var_price`. 

 - Your answers should be rounded to 1 decimal place.   

In [3]:
# Use this cell to write your code for Task 3
# Grouping by bedrooms and calculating mean and variance
price_by_rooms = df.groupby('bedrooms')['sale_price'].agg(
    avg_price=lambda x: round(x.mean(), 1),
    var_price=lambda x: round(x.var(), 1)
).reset_index()
price_by_rooms

,bedrooms,avg_price,var_price
0,2,67076.4,5.652896e+08
1,3,154665.1,2.378289e+09
2,4,234704.6,1.725211e+09
3,5,301515.9,2.484328e+09
4,6,375741.3,3.924432e+09


# Task 4

Fit a baseline model to predict the sale price of a house.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “validation.csv” to predict new values based on your model. You must return a dataframe named `base_result`, that includes `house_id` and `price`. The price column must be your predicted values.

In [4]:
# Use this cell to write your code for Task 4
# Loading the "train.csv" and "validation.csv"
train_df = pd.read_csv("train.csv")
val_df = pd.read_csv("validation.csv")

# Computing the mean baseline sale price 
baseline_mean_price = train_df['sale_price'].mean()

# Predicting the mean sale price for the validation set
base_result = pd.DataFrame({
    'house_id': val_df['house_id'],
    'price': round(baseline_mean_price, 1)  # Rounded to 1 decimal place (optional)
})
base_result.head()

,house_id,price
0,1331375,225844.7
1,1630115,225844.7
2,1645745,225844.7
3,1336775,225844.7
4,1888274,225844.7


# Task 5

Fit a comparison model to predict the sale price of a house.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “validation.csv” to predict new values based on your model. You must return a dataframe named `compare_result`, that includes `house_id` and `price`. The price column must be your predicted values.

In [5]:
# Use this cell to write your code for Task 5
# Loading the "train.csv" and "validation.csv"
train_df = pd.read_csv("train.csv")
val_df = pd.read_csv("validation.csv")

# Defining features and target
features = ['city', 'months_listed', 'bedrooms', 'house_type', 'area'] 
target = 'sale_price'

# Preprocessing features
num_features = ['months_listed', 'area', 'bedrooms']
cat_features = ['city', 'house_type']

numeric_transformer = SimpleImputer(strategy='mean')

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_features),
        ('cat', categorical_transformer, cat_features)
    ])

# Building a pipeline with a model
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# Fitting the model
model.fit(train_df[features], train_df[target])

# Predicting on validation data
predictions = model.predict(val_df[features])

# Creating output DataFrame
compare_result = pd.DataFrame({
    'house_id': val_df['house_id'],
    'price': predictions
})

# Optional: round price
compare_result['price'] = compare_result['price'].round(1)

# Show preview
compare_result.head()



,house_id,price
0,1331375,121527.8
1,1630115,304386.6
2,1645745,384760.1
3,1336775,123976.3
4,1888274,271186.2


In [6]:
import os
print(os.getcwd())


C:\Users\MAYKISH\house_sales_1


In [9]:
print(os.path.abspath("house_sales.ipynb"))


C:\Users\MAYKISH\house_sales_1\house_sales.ipynb
